<a href="https://colab.research.google.com/github/Brendon1109/MACHINE-LEARNING/blob/main/26784769.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=ac7493b99da8259627ce260aa2126fef66c693cf236769bfa8c58b62cbb01dc4
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np

! wget https://storage.googleapis.com/bdt-spark-store/external_sources.csv -O gcs_external_sources.csv
! wget https://storage.googleapis.com/bdt-spark-store/internal_data.csv -O gcs_internal_data.csv

df_data = pd.read_csv('gcs_internal_data.csv')
df_ext = pd.read_csv('gcs_external_sources.csv')

df_full = df_data.merge(df_ext, on='SK_ID_CURR', how='inner')
df_full.head()

columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                  'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE',
                  'DAYS_ID_PUBLISH', 'CODE_GENDER', 'AMT_ANNUITY',
                  'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                  'ORGANIZATION_TYPE', 'DAYS_LAST_PHONE_CHANGE',
                  'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']
df = df_full[columns_extract]
df.head(3)

# set the seed for reproducibility
np.random.RandomState(101)
train, test = np.split(df.sample(frac=1), [int(.8*len(df))])
print(train.TARGET.value_counts()/len(train.index))
print(test.TARGET.value_counts()/len(test.index))
#Handle the categorical variables

train = pd.get_dummies(train)
test = pd.get_dummies(test)

print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)

# Align the training and testing data, keep only columns present in both dataframes
train, test = train.align(test, join = 'inner', axis = 1)

print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)
#Get labels from data

train_labels = train['TARGET']
test_labels = test['TARGET']
#Fill in missing data and scale

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer as Imputer

# Drop the target from the training data
if 'TARGET' in train:
    train = train.drop(columns = ['TARGET'])
    test = test.drop(columns = ['TARGET'])
else:
    train = train.copy()
    test = test.copy()

# Feature names
features = list(train.columns)

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = StandardScaler()

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(test)

scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)
#Fit random forest

from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100,
                                       random_state = 50,
                                       verbose = 1, n_jobs = -1)
# Train on the training data
random_forest.fit(train, train_labels)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict(test)
#Evaluate on test

from sklearn.metrics import accuracy_score, roc_auc_score

print(accuracy_score(test_labels, predictions))
feature_importances.sort_values('importance', ascending=False).head(10)

--2023-11-05 06:35:44--  https://storage.googleapis.com/bdt-spark-store/external_sources.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.207, 142.250.153.207, 74.125.128.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15503836 (15M) [text/csv]
Saving to: ‘gcs_external_sources.csv’

gcs_external_source 100%[===================>]  14.79M  --.-KB/s    in 0.1s    

2023-11-05 06:35:44 (129 MB/s) - ‘gcs_external_sources.csv’ saved [15503836/15503836]

--2023-11-05 06:35:44--  https://storage.googleapis.com/bdt-spark-store/internal_data.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.207, 142.250.153.207, 74.125.128.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 152978396 (146M) [text/csv]
Saving to: ‘gcs_internal_d

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   41.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.5min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    1.2s


0.9192071931450498


[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    2.2s finished


,feature,importance
1,EXT_SOURCE_2,0.098178
2,EXT_SOURCE_3,0.086283
3,DAYS_BIRTH,0.077291
5,DAYS_ID_PUBLISH,0.076094
7,DAYS_REGISTRATION,0.075636
6,AMT_ANNUITY,0.071110
10,DAYS_LAST_PHONE_CHANGE,0.067675
4,DAYS_EMPLOYED,0.065287
9,AMT_CREDIT,0.065107
11,AMT_INCOME_TOTAL,0.056438


In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, Imputer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

# Initialize Spark session
spark = SparkSession.builder \
    .appName("RandomForestClassifier") \
    .getOrCreate()

# Load data
internal_data_path = "gcs_internal_data.csv"
external_sources_path = "gcs_external_sources.csv"

df_data = spark.read.option("header", "true").csv(internal_data_path)
df_ext = spark.read.option("header", "true").csv(external_sources_path)

# Merge dataframes on 'SK_ID_CURR' column
df_full = df_data.join(df_ext, 'SK_ID_CURR', 'inner')

# Select relevant columns
columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                   'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE',
                   'DAYS_ID_PUBLISH', 'CODE_GENDER', 'AMT_ANNUITY',
                   'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                   'ORGANIZATION_TYPE', 'DAYS_LAST_PHONE_CHANGE',
                   'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']

df = df_full.select(columns_extract)

# Convert "TARGET" column to DoubleType
df = df.withColumn("TARGET", col("TARGET").cast("double"))

# Convert columns to DoubleType
for col_name in ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'AMT_ANNUITY',
                 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE']:
    df = df.withColumn(col_name, col(col_name).cast("double"))

# Handle categorical variables using StringIndexer and OneHotEncoder
categorical_cols = ['NAME_EDUCATION_TYPE', 'CODE_GENDER', 'ORGANIZATION_TYPE', 'NAME_INCOME_TYPE']

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=indexer.getOutputCol(), outputCol=indexer.getOutputCol() + "_encoded") for indexer in indexers]

# Impute missing values
imputer = Imputer(strategy="mean", inputCols=['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
                                              'AMT_ANNUITY', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE',
                                              'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE',
                                              'AMT_INCOME_TOTAL', 'OWN_CAR_AGE'],
                  outputCols=['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
                              'AMT_ANNUITY', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE',
                              'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE',
                              'AMT_INCOME_TOTAL', 'OWN_CAR_AGE'])

assembler_inputs = [encoder.getOutputCol() for encoder in encoders] + ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH',
                                                                       'AMT_ANNUITY', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE',
                                                                       'AMT_CREDIT', 'DAYS_LAST_PHONE_CHANGE', 'AMT_INCOME_TOTAL',
                                                                       'OWN_CAR_AGE']

assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# Create stages for the pipeline
stages = indexers + encoders + [imputer, assembler]

# Create pipeline
pipeline = Pipeline(stages=stages)

# Fit the pipeline to the data
pipeline_model = pipeline.fit(df)
df = pipeline_model.transform(df)

# Split data into train and test sets
train, test = df.randomSplit([0.8, 0.2], seed=12345)

# Initialize and train Random Forest classifier
rf = RandomForestClassifier(labelCol="TARGET", featuresCol="features", numTrees=100, seed=12345)
model = rf.fit(train)

# Make predictions on the test data
predictions = model.transform(test)

# Evaluate the model using accuracy metric
evaluator = BinaryClassificationEvaluator(labelCol="TARGET")
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

print("Accuracy: ", accuracy)

# Save the model to disk
model_path = "random_forest_model"
model.write().overwrite().save(model_path)

# Stop the Spark session
spark.stop()


Accuracy:  0.6427403822986383


In [2]:
!pip install koalas

In [4]:
import sys
print(sys.version)


3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [5]:
!pip install --upgrade koalas


In [ ]:
import pandas as pd
import numpy as np
import databricks.koalas as ks
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# Load data using Koalas
df_data = ks.read_csv('gcs_internal_data.csv')
df_ext = ks.read_csv('gcs_external_sources.csv')

# Merge dataframes on 'SK_ID_CURR' column
df_full = df_data.merge(df_ext, on='SK_ID_CURR', how='inner')

# Select relevant columns
columns_extract = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                   'DAYS_BIRTH', 'DAYS_EMPLOYED', 'NAME_EDUCATION_TYPE',
                   'DAYS_ID_PUBLISH', 'CODE_GENDER', 'AMT_ANNUITY',
                   'DAYS_REGISTRATION', 'AMT_GOODS_PRICE', 'AMT_CREDIT',
                   'ORGANIZATION_TYPE', 'DAYS_LAST_PHONE_CHANGE',
                   'NAME_INCOME_TYPE', 'AMT_INCOME_TOTAL', 'OWN_CAR_AGE', 'TARGET']

df = df_full[columns_extract]

# Handle categorical variables using one-hot encoding
df = pd.get_dummies(df, columns=['NAME_EDUCATION_TYPE', 'CODE_GENDER', 'ORGANIZATION_TYPE', 'NAME_INCOME_TYPE'])

# Set the seed for reproducibility
np.random.seed(101)
train, test = np.split(df.sample(frac=1), [int(.8*len(df))])

# Get labels from data
train_labels = train['TARGET']
test_labels = test['TARGET']

# Drop the target from the training data
train_features = train.drop(columns=['TARGET'])
test_features = test.drop(columns=['TARGET'])

# Median imputation of missing values and scaling
imputer = StandardScaler()
train_features = imputer.fit_transform(train_features)
test_features = imputer.transform(test_features)

# Fit random forest
random_forest = RandomForestClassifier(n_estimators=100, random_state=50, n_jobs=-1)
random_forest.fit(train_features, train_labels)

# Make predictions on the test data
predictions = random_forest.predict(test_features)

# Evaluate on test
print("Accuracy Score:", accuracy_score(test_labels, predictions))
print("ROC AUC Score:", roc_auc_score(test_labels, predictions))
